In [ ]:
import csv
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import numpy as np
from nltk.corpus import sentiwordnet as swn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
'''
import twitter
api = twitter.Api(consumer_key='1JyxFD77cbNWFtuHPg9Qcw8I9',
                 consumer_secret='8DUwmZXezxS7FYGy6S7bfKN1HsxksmxJ4A4qVw4Wy1MAXYMfJE',
                 access_token_key='913162907629780992-k7BVVkhLCPX3xX0KNiKob3UfQjr09od',
                 access_token_secret='ELXVySL3kyxkkdEXsjXdhW7roDtNuWGaiENveCt3qVMTD')
print api.VerifyCredentials()

def createTestData(search_string):
    try:
        tweets_fetched = api.GetSearch(search_string,count=100)
        print "Great! we fetched "+str(len(tweets_fetched))+" tweets "+"on "+search_string
        return [{"text":status.text,"label":None} for status in tweets_fetched]
    except:
        print "Sorry!! no match"
        return None
#search_string = raw_input("Enter your search pattern")
#testData = createTestData(search_string)'''

#Function to create Training Data
def createTrainingData(corpusFile):
    trainingData=[]
    with open(corpusFile,'rb') as tsvFile:
        lineReader = csv.reader(tsvFile,dialect='excel-tab')
        print lineReader
        for row in lineReader:
            if len(row) == 2:
                sub_row = row[0].split("  ")
                trainingData.append({"tweet-id":sub_row[0],"label":sub_row[1],"tweet_text":row[1]})
            else:
                trainingData.append({"tweet-id":row[0],"label":row[1],"tweet_text":row[2]})
    return trainingData
trainingData = createTrainingData("G:\\Doit\\UC_ACAD\\CS Courses\\NLP\\semEvalProject\\semval-task4-training data\\semval-task4-training data\\2016downloaded4-subtask A.tsv")

#PreProcessing of tweets using regular expression and nltk modules
class PreProcessTweets:
    def __init__(self):
        #taking stopwords like a,an,the,by,punctuations,url,@symbols..
        self._stopwords = set(stopwords.words('english')+list(punctuation)+['AT_USER','URL'])
    def processTweets(self,list_of_tweets):
        processedTweets=[]
        #print list_of_tweets
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet['tweet_text']),tweet['label']))
        return processedTweets
    def _processTweet(self,tweet):
        #processing tweets as below
        # 1. convert all to lower cases
        tweet = tweet.lower()
        #2. Replace links with the word with "URL" so as to avoid stopwords
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
        #3. Replace @username with "AT_USER" so as to avoid as stopwords
        tweet = re.sub('@[^\s]+','AT_USER',tweet)
        #4. Remove # tags from words , eg: #love --> love
        tweet = re.sub(r'#([^\s])+',r'\1',tweet)
        #5.convert long words to short format
        tweet=re.sub(r'(.?)\1+', r'\1\1', tweet) 
        #6. Do Word tokenize
        tweet = word_tokenize(tweet)
        return [word for word in tweet if word not in self._stopwords]
tweetProcessor = PreProcessTweets()
PreProcessedTrainingData = tweetProcessor.processTweets(trainingData)

#SVM Classifier Training
svmTrainingData = [' '.join(tweet[0]) for tweet in PreProcessedTrainingData]
vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit_transform(svmTrainingData)
vocabulary = vectorizer.get_feature_names()
swn_weights = []
for word in vocabulary:
    #adding weights to the features by taking pos_score and neg_scores of word
    try:
        synset = list(swn.senti_synsets(word)) #taking synonyms of word
        common_meaning = synset[0]
        if common_meaning.pos_score() > common_meaning.neg_score():
            weight = pos_score
        elif common_meaning.pos_score() < common_meaning.neg_score():
            weight = neg_score
        else:
            weight = 0
    except:
        weight =0
    swn_weights.append(weight)
swn_X = []
for row in X:
    swn_X.append(np.multiply(row,np.array(swn_weights)))
swn_X = np.vstack(swn_X)

#Lets map positive to 1,negative to 2
labels_array={'positive':1,'negative':2}
labels=[labels_array[tweet[1]] for tweet in PreProcessedTrainingData]
y = np.array(labels)

#build SVM classifier
SVMClassifier = SVC()
SVMClassifier.fit(swn_X,y)

In [ ]:
swn_X